In [ ]:
!pip install paddleocr
!pip install paddlepaddle
!git clone https://github.com/PaddlePaddle/PaddleOCR

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 389 kB 4.9 MB/s 
     |████████████████████████████████| 2.3 MB 38.5 MB/s 
     |████████████████████████████████| 604 kB 48.6 MB/s 
     |████████████████████████████████| 4.9 MB 34.8 MB/s 
     |████████████████████████████████| 399 kB 36.7 MB/s 
     |████████████████████████████████| 104 kB 49.2 MB/s 
     |████████████████████████████████| 204 kB 45.5 MB/s 
     |████████████████████████████████| 2.3 MB 42.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 112.5 MB 58 kB/s 
     |████████████████████████████████| 394 kB 49.7 MB/s 
Cloning into 'PaddleOCR'...
remote: Enumerating objects: 42497, done.
remote: Counting objects: 100% (737/737), done.
remote: Compressing objects: 100% (388/388), done.
remote: Total 42497 (delta 422), reused 614 (delta 346

In [ ]:
!gdown 1JR3Sgz2-AyfIDT7jYRXL8XMrMom57DAi
!gdown 13du4J0NVPa6NWcehJITx1jcfYsINdBE5

Downloading...
From: https://drive.google.com/uc?id=1JR3Sgz2-AyfIDT7jYRXL8XMrMom57DAi
To: /content/english_dataset.csv
100% 643k/643k [00:00<00:00, 76.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=13du4J0NVPa6NWcehJITx1jcfYsINdBE5
To: /content/Allergen.csv
100% 2.05k/2.05k [00:00<00:00, 2.06MB/s]


In [ ]:
import re
from paddleocr import PaddleOCR, draw_ocr
import spacy
import en_core_web_sm
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
np.set_printoptions(precision=2)

In [ ]:
ocr_model = PaddleOCR(lang='en',use_angle_cls=True)

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3.86M/3.86M [00:13<00:00, 287kiB/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer/en_PP-OCRv3_rec_infer.tar


100%|██████████| 9.96M/9.96M [00:12<00:00, 766kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:13<00:00, 168kiB/s]

[2022/09/09 11:45:56] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='f

In [ ]:
df = pd.read_csv('english_dataset.csv')
aller_list = pd.read_csv('Allergen.csv') 

In [ ]:
df = df.drop(414)

In [ ]:
df.head()

,ingredients_text,allergens,image_ingredients_url,image_nutrition_url,is_allergen,ingredeint_language
0,"water, _almonds_* 5%, agave syrup*, sea salt",en:nuts,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,True,en
1,"Wheat flour, sugar, almonds 18%, eggs, egg Whi...","en:eggs,en:gluten,en:milk,en:nuts,en:soybeans",https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,True,en
2,"_WHEAT_ flour, water, tomato pulp, mozzarella ...","en:gluten,en:milk",https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,True,en
3,LATTE_ INTERO BIOLOGICO,en:milk,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,True,en
4,"Pistachio 45%, sugar, milk powder, extra virgi...","en:milk,en:nuts,en:soybeans",https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,True,en


In [ ]:
nlp = en_core_web_sm.load()

In [ ]:
aller_list.head()

,Ingridients
0,celery
1,lupin
2,lupine
3,milk
4,cheese


In [ ]:
allergen_embeddings = []
for allergen in aller_list.Ingridients:
  vector = nlp(allergen)
  allergen_embeddings.append(vector.vector)

In [ ]:
def preprocess(text):
    text = re.sub('#S+', '',text)
    text = re.sub('@S+', '  ',text)
    text = re.sub('httpS+s*', ' ', text) 
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+-,./:;<=>?@[]^_`{|}~"""), ' ', text)
    text = re.sub('RT|cc', ' ', text)
    text = re.sub(r'<[^<]+?>', '', text)
    text = re.sub(r'[0-9]', "", text)
    text = text.replace('\n', ' ') 
    text = re.sub(r'\s+', ' ', text)
    return text

In [ ]:
def detect_allergen(image):
  allergen = []
  final_ocr = []
  image_ocr = ocr_model.ocr(image)
  for item in image_ocr:
    final_ocr.append(item[1][0])
  final_ocr = " ".join(str(x) for x in final_ocr)
  clean_ocr = preprocess(final_ocr)
  clean_ocr = word_tokenize(clean_ocr)
  clean_ocr = [i.lower() for i in clean_ocr]
  print(list(clean_ocr))
  for i in clean_ocr:
    for j in allergen_embeddings:
      ingred = nlp(i)
      ingred = np.array(ingred.vector.reshape(1, -1))
      aller = np.array(j).reshape(1, -1)
      sim = cosine_similarity(ingred, aller)
      if sim > 0.90:


        print('May contain {}'.format(i))

In [ ]:
detect_allergen(df.image_ingredients_url[10])

100%|██████████| 24.7k/24.7k [00:00<00:00, 4.19MiB/s]

[2022/09/09 11:47:32] ppocr DEBUG: dt_boxes num : 5, elapse : 0.04652667045593262
[2022/09/09 11:47:32] ppocr DEBUG: cls num  : 5, elapse : 0.04334402084350586


[2022/09/09 11:47:34] ppocr DEBUG: rec_res num  : 5, elapse : 1.942516803741455
['gbbiscuitsfilledwith', 'cappuccinocream', 'ngredientswheat', 'lour', 'sugar', 'vegetable', 'os', 'of', 'pam', 'and', 'sunflower', 'vegetablemargrine', 'pam', 'oilwaterwhole', 'mik', 'powder', 'wheat', 'starchglucose', 'syrup', 'hazelnut', 'paste', 'eggs', 'butter', 'whey', 'powderffe', 'in', 'the', 'contain', 'peanut', 'traces', 'no', 'hydrogenated', 'fats', 'keep', 'cool', 'and', 'dry', 'away', 'from', 'light', 'sources', 'best', 'before', 'end', 'see', 'date', 'printed', 'below', 'madeinitaly']
May contain wheat
May contain hazelnut
May contain butter
May contain peanut


In [ ]:
detect_allergen(df.image_ingredients_url[1])

100%|██████████| 9.54k/9.54k [00:00<00:00, 2.46MiB/s]

[2022/08/30 13:41:28] ppocr DEBUG: dt_boxes num : 3, elapse : 0.04230809211730957
[2022/08/30 13:41:28] ppocr DEBUG: cls num  : 3, elapse : 0.026709794998168945


[2022/08/30 13:41:29] ppocr DEBUG: rec_res num  : 3, elapse : 0.6613740921020508
['almond', 'cookies', 'confectionary', 'bakery', 'product', 'ingredients', 'wheatavours', 'salt', 'contains', 'wheat', 'eggs', 'pine', 'nuts', 'almonds', 'milk', 'may', 'contain', 'traces', 'of', 'soy', 'store', 'dry', 'and', 'cool', 'plac']
May contain almond
May contain wheat
May contain pine
May contain nuts
May contain milk
May contain milk
May contain soy


In [ ]:
detect_allergen(df.image_ingredients_url[15])

100%|██████████| 7.43k/7.43k [00:00<00:00, 1.58MiB/s]

[2022/08/30 13:43:44] ppocr DEBUG: dt_boxes num : 1, elapse : 0.03954625129699707
[2022/08/30 13:43:44] ppocr DEBUG: cls num  : 1, elapse : 0.011813879013061523


[2022/08/30 13:43:44] ppocr DEBUG: rec_res num  : 1, elapse : 0.134002685546875
['wheat', 'semollna', 'water']
May contain wheat


In [ ]:
detect_allergen(df.image_ingredients_url[1000])

100%|██████████| 36.6k/36.6k [00:00<00:00, 5.85MiB/s]

[2022/08/30 13:13:21] ppocr DEBUG: dt_boxes num : 17, elapse : 0.1138145923614502


[2022/08/30 13:13:21] ppocr DEBUG: cls num  : 17, elapse : 0.14037442207336426
[2022/08/30 13:13:28] ppocr DEBUG: rec_res num  : 17, elapse : 6.499310255050659
['ngredients', 'driedpotatoesvegetableoilicontainsoneorvoreofthefollowingcorn', 'oil', 'cottonseed', 'oil', 'soybean', 'oiland', 'or', 'sunflower', 'oil', 'degerminated', 'yellowcorn', 'flour', 'cornstarch', 'rice', 'flour', 'maltodextrin', 'mono', 'and', 'diglycerides', 'contains', 'or', 'less', 'of', 'salt', 'dextrose', 'monosodium', 'glutamate', 'whey', 'onion', 'powder', 'citricacid', 'spices', 'natural', 'and', 'a', 'ificial', 'flavors', 'autolyzed', 'yeast', 'extract', 'tomatopowdergarlicpowder', 'torulayeast', 'sugar', 'lacticacid', 'sodiumdiacetate', 'disodiuminosinate', 'disodium', 'guanylate', 'paprika', 'extract', 'color', 'wheat', 'starchcontainsmilkand', 'wheatingredientsingredientes', 'papas', 'deshidratadas', 'aceite', 'vegetal', 'contiene', 'al', 'menos', 'uno', 'de', 'lossiguientes', 'aceites', 'maizsemilla', 'd

In [ ]:
detect_allergen(df.image_ingredients_url[100])

100%|██████████| 5.68k/5.68k [00:00<00:00, 1.58MiB/s]

[2022/08/30 13:16:13] ppocr DEBUG: dt_boxes num : 0, elapse : 0.017305612564086914
[2022/08/30 13:16:13] ppocr DEBUG: cls num  : 0, elapse : 0
[2022/08/30 13:16:13] ppocr DEBUG: rec_res num  : 0, elapse : 3.337860107421875e-06
[]


In [ ]:
detect_allergen(df.image_ingredients_url[200])

100%|██████████| 30.3k/30.3k [00:00<00:00, 1.72MiB/s]

[2022/08/30 13:16:14] ppocr DEBUG: dt_boxes num : 17, elapse : 0.1301126480102539


[2022/08/30 13:16:14] ppocr DEBUG: cls num  : 17, elapse : 0.15755200386047363
[2022/08/30 13:16:16] ppocr DEBUG: rec_res num  : 17, elapse : 2.2917749881744385
['baiesdegojilotabibica', 'consormmer', 'e', 'preference', 'ant', 'fin', 'a', 'grammes', 'de', 'preference', 'le', 'matinplaneteyaourou', 'dpides', 'g', 'glucides', 'gdont', 'sucre', 'gpoidsnet', 'francestcerife', 'sans', 'modification', 'genetique']


In [ ]:
detect_allergen(df.image_ingredients_url[20])

100%|██████████| 26.3k/26.3k [00:00<00:00, 3.50MiB/s]


[2022/08/30 13:16:56] ppocr DEBUG: dt_boxes num : 14, elapse : 0.18452119827270508
[2022/08/30 13:16:57] ppocr DEBUG: cls num  : 14, elapse : 0.23568272590637207
[2022/08/30 13:16:59] ppocr DEBUG: rec_res num  : 14, elapse : 2.402991771697998
['nouvelle', 'recettedents', 'puus', 'blancheshaleinepurem', 'avec', 'edulcorants', 'parfum', 'menthgltcerodlorantsdioxydaricttsegperfetineednsa']


In [ ]:
detect_allergen(df.image_ingredients_url[1200])

100%|██████████| 5.91k/5.91k [00:00<00:00, 2.86MiB/s]

[2022/08/30 13:18:29] ppocr DEBUG: dt_boxes num : 1, elapse : 0.01664900779724121
[2022/08/30 13:18:29] ppocr DEBUG: cls num  : 1, elapse : 0.013390779495239258
[2022/08/30 13:18:29] ppocr DEBUG: rec_res num  : 1, elapse : 0.08738374710083008


['chocolatno']


In [ ]:
detect_allergen(df.image_ingredients_url[30])

100%|██████████| 10.5k/10.5k [00:00<00:00, 2.97MiB/s]

[2022/08/30 13:19:03] ppocr DEBUG: dt_boxes num : 2, elapse : 0.02733469009399414
[2022/08/30 13:19:03] ppocr DEBUG: cls num  : 2, elapse : 0.01934814453125


[2022/08/30 13:19:03] ppocr DEBUG: rec_res num  : 2, elapse : 0.43738889694213867
['pates', 'alimentaires', 'ingredients', 'semoule', 'de', 'ble', 'dur', 'eauconserver', 'dans', 'un', 'endroit', 'frais', 'et', 'sec']


In [ ]:
detect_allergen(df.image_ingredients_url[50])

100%|██████████| 7.03k/7.03k [00:00<00:00, 2.00MiB/s]

[2022/08/30 13:19:59] ppocr DEBUG: dt_boxes num : 0, elapse : 0.029948711395263672
[2022/08/30 13:19:59] ppocr DEBUG: cls num  : 0, elapse : 0
[2022/08/30 13:19:59] ppocr DEBUG: rec_res num  : 0, elapse : 3.0994415283203125e-06
[]


In [ ]:
detect_allergen(df.image_ingredients_url[40])

100%|██████████| 21.1k/21.1k [00:00<00:00, 1.58MiB/s]

[2022/08/30 13:20:10] ppocr DEBUG: dt_boxes num : 9, elapse : 0.1728811264038086


[2022/08/30 13:20:11] ppocr DEBUG: cls num  : 9, elapse : 0.07375717163085938
[2022/08/30 13:20:12] ppocr DEBUG: rec_res num  : 9, elapse : 1.1981556415557861
['pastaalimenticia', 'de', 'calidad', 'superioringredientssemoulede', 'bledur', 'gluteningredlentsdurumwheat', 'semolina', 'gluteningredlentssemola', 'de', 'trigo', 'duro', 'gluten', 'npuede', 'contener', 'trazas', 'de', 'huevomay', 'contain', 'traces', 'of', 'egggeatencion', 'alconsumidor']
May contain semolina
May contain gluten


In [ ]:
detect_allergen(df.image_ingredients_url[45])

100%|██████████| 28.9k/28.9k [00:00<00:00, 4.07MiB/s]

[2022/08/30 13:21:21] ppocr DEBUG: dt_boxes num : 9, elapse : 0.1370835304260254


[2022/08/30 13:21:21] ppocr DEBUG: cls num  : 9, elapse : 0.07544517517089844
[2022/08/30 13:21:23] ppocr DEBUG: rec_res num  : 9, elapse : 1.7384681701660156
['ingredients', 'roasted', 'peanuts', 'whole', 'grain', 'rolledoats', 'sugar', 'chicory', 'root', 'extract', 'glucose', 'syrupwheat', 'flour', 'vegetable', 'fats', 'palm', 'shea', 'maltodextrincocoa', 'mass', 'almond', 'pieces', 'fructose', 'fat', 'reduce', 'ocoa', 'powder', 'rice', 'flour', 'sunflower', 'oil', 'lactose', 'milk', 'peanut', 'hutter', 'humectant', 'glycerolsalt', 'emulifierssunflower', 'and', 'soy', 'lecithindextrose', 'natural', 'flayouringswheat', 'germcocoa', 'butter', 'antioxidant', 'tocopherelricextract', 'honey']
May contain flour
May contain shea
May contain almond
May contain flour
May contain lactose
May contain milk
May contain milk
May contain peanut
May contain soy
May contain butter
